In [1]:
import pandas as pd
from selenium import webdriver
from tqdm import tqdm

In [2]:
DATA_DIR = './data'

In [3]:
stop_df = pd.read_csv(DATA_DIR + '/pre_stop_points.csv')
stop_df.activity = stop_df.pop('activity_class')

near_poi_df = pd.read_csv(DATA_DIR + '/pre_nearest_POIs.csv')
near_poi_df = near_poi_df.drop_duplicates()
near_poi_df = near_poi_df.reset_index(drop=True)

In [4]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('--lang=ko')
options.add_argument('window-size=1920,1080')

In [5]:
shotest_dist_list = list()

for i in tqdm(stop_df.index):
    origin = f'{stop_df.lng[i]},{stop_df.lat[i]}'
    _near_poi_df = near_poi_df[near_poi_df.id == stop_df.id[i]]

    for j in _near_poi_df.index:
        try:
            departure = f'{_near_poi_df.lng[j]},{_near_poi_df.lat[j]}'
            url = f'https://map.naver.com/v5/directions/{origin},,,/{departure},,,/-/walk'

            driver = webdriver.Chrome('./chromedriver', options=options)
            driver.implicitly_wait(3.0)
            driver.get(url)
        except:
            shotest_dist_list.append([j, '-1'])
            continue
            
        try:
            shotest_dist = driver.find_element_by_class_name('summary_text').text
            shotest_dist_list.append([j, shotest_dist])
        except:
            error_area = driver.find_elements_by_class_name('directions_error_area')

            if len(error_area) != 0:
                ng_start = error_area[0].find_elements_by_class_name('ng-star-inserted')[0].text

                if '50km' in ng_start:
                    shotest_dist_list.append([j,'50km'])
                elif '동일' in ng_start:
                    shotest_dist_list.append([j, '0m'])
            else:
                shotest_dist_list.append([j, '-1'])
        driver.close()

100%|████████████████████████████████████████████████████████████████████████████| 907/907 [29:52:26<00:00, 124.92s/it]


In [6]:
shotest_dist_df = pd.DataFrame(shotest_dist_list, columns=['index', 'shotest_dist'])
shotest_dist_df.to_csv(DATA_DIR + '/tmp_shotest_dist.csv', index=False)